# Avaliação do Dataset Weight-Height

Este notebook apresenta a análise completa e visualizações dos resultados obtidos com o dataset Weight-Height, incluindo:
- Classificação de Gênero (Supervisionado)
- Análise de Clusters (Não Supervisionado)
- Métricas de Avaliação
- Comparações entre Modelos

In [ ]:
# Importações
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.decomposition import PCA
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    silhouette_score, davies_bouldin_score
)
import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("✓ Bibliotecas carregadas")

## 1. Geração e Exploração dos Dados

In [ ]:
# Gerar dados sintéticos
np.random.seed(42)
n_samples = 10000

# Homens
n_male = n_samples // 2
male_height = np.random.normal(178, 7, n_male)
male_weight = np.random.normal(85, 12, n_male)

# Mulheres
n_female = n_samples - n_male
female_height = np.random.normal(163, 6, n_female)
female_weight = np.random.normal(65, 10, n_female)

# DataFrame
df = pd.DataFrame({
    'Gender': ['Male'] * n_male + ['Female'] * n_female,
    'Height': np.concatenate([male_height, female_height]),
    'Weight': np.concatenate([male_weight, female_weight])
})

df = df.sample(frac=1, random_state=42).reset_index(drop=True)

print(f"Dataset Weight-Height")
print(f"Amostras: {len(df)}")
print(f"Features: Height (cm), Weight (kg)")
print(f"\nDistribuição:")
print(df['Gender'].value_counts())

print(f"\nEstatísticas:")
print(df.groupby('Gender')[['Height', 'Weight']].describe())

df.head(10)

In [ ]:
# Visualização exploratória
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Scatter Height vs Weight
for gender, color in [('Male', 'blue'), ('Female', 'red')]:
    data = df[df['Gender'] == gender]
    axes[0, 0].scatter(data['Height'], data['Weight'], 
                       c=color, alpha=0.4, s=10, label=gender)
axes[0, 0].set_xlabel('Altura (cm)', fontweight='bold')
axes[0, 0].set_ylabel('Peso (kg)', fontweight='bold')
axes[0, 0].set_title('Altura vs Peso', fontsize=14, fontweight='bold')
axes[0, 0].legend()
axes[0, 0].grid(alpha=0.3)

# Histograma Altura
axes[0, 1].hist([df[df['Gender']=='Male']['Height'],
                 df[df['Gender']=='Female']['Height']],
                label=['Male', 'Female'], alpha=0.7, bins=50)
axes[0, 1].set_xlabel('Altura (cm)', fontweight='bold')
axes[0, 1].set_ylabel('Frequência', fontweight='bold')
axes[0, 1].set_title('Distribuição de Altura', fontsize=14, fontweight='bold')
axes[0, 1].legend()
axes[0, 1].grid(axis='y', alpha=0.3)

# Histograma Peso
axes[1, 0].hist([df[df['Gender']=='Male']['Weight'],
                 df[df['Gender']=='Female']['Weight']],
                label=['Male', 'Female'], alpha=0.7, bins=50)
axes[1, 0].set_xlabel('Peso (kg)', fontweight='bold')
axes[1, 0].set_ylabel('Frequência', fontweight='bold')
axes[1, 0].set_title('Distribuição de Peso', fontsize=14, fontweight='bold')
axes[1, 0].legend()
axes[1, 0].grid(axis='y', alpha=0.3)

# Boxplots
df.boxplot(column=['Height', 'Weight'], by='Gender', ax=axes[1, 1])
axes[1, 1].set_title('Boxplots por Gênero', fontsize=14, fontweight='bold')
axes[1, 1].set_xlabel('Medida', fontweight='bold')
plt.suptitle('')

plt.tight_layout()
plt.savefig('weight_height_exploratory.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Análise exploratória concluída")

## 2. Aprendizado Supervisionado

In [ ]:
# Preparar dados
X = df[['Height', 'Weight']].values
y = np.where(df['Gender'] == 'Male', 1, 0)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Modelos
models = {
    "Decision Tree": DecisionTreeClassifier(random_state=42),
    "Random Forest": RandomForestClassifier(random_state=42, n_estimators=100),
    "SVM": SVC(random_state=42),
    "Naive Bayes": GaussianNB(),
    "K-NN": KNeighborsClassifier(n_neighbors=5),
    "Logistic Regression": LogisticRegression(max_iter=1000, random_state=42)
}

results = []

for name, model in models.items():
    model.fit(X_train_scaled, y_train)
    y_pred = model.predict(X_test_scaled)
    
    results.append({
        'Modelo': name,
        'Accuracy': accuracy_score(y_test, y_pred),
        'Precision': precision_score(y_test, y_pred),
        'Recall': recall_score(y_test, y_pred),
        'F1-Score': f1_score(y_test, y_pred)
    })

df_results = pd.DataFrame(results)
print("\nResultados Supervisionados:")
print(df_results.round(4).to_string(index=False))

df_results.to_csv('weight_height_supervised_results.csv', index=False)
print("\n✓ Resultados salvos")

In [ ]:
# Visualização
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
metrics = ['Accuracy', 'Precision', 'Recall', 'F1-Score']

for idx, metric in enumerate(metrics):
    ax = axes[idx // 2, idx % 2]
    bars = ax.bar(df_results['Modelo'], df_results[metric], color='lightyellow', edgecolor='black')
    
    ax.axhline(y=0.70, color='red', linestyle='--', linewidth=2, label='Meta 70%')
    
    for bar, value in zip(bars, df_results[metric]):
        if value >= 0.70:
            bar.set_color('green')
            bar.set_alpha(0.7)
    
    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height + 0.01,
                f'{height:.3f}', ha='center', va='bottom', fontsize=10, fontweight='bold')
    
    ax.set_ylabel(metric, fontsize=12, fontweight='bold')
    ax.set_title(f'{metric} - Weight-Height Dataset', fontsize=14, fontweight='bold')
    ax.set_xticklabels(df_results['Modelo'], rotation=45, ha='right')
    ax.set_ylim(0, 1.1)
    ax.legend()
    ax.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.savefig('weight_height_supervised_metrics.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Visualização salva")

## 3. Aprendizado Não Supervisionado

In [ ]:
# Clustering
X_scaled = StandardScaler().fit_transform(X)

# K-Means
kmeans = KMeans(n_clusters=2, random_state=42)
clusters_km = kmeans.fit_predict(X_scaled)
sil_km = silhouette_score(X_scaled, clusters_km)
db_km = davies_bouldin_score(X_scaled, clusters_km)

print(f"K-Means:")
print(f"  Silhouette: {sil_km:.4f}")
print(f"  Davies-Bouldin: {db_km:.4f}")

# Hierarchical
hier = AgglomerativeClustering(n_clusters=2, linkage='average')
clusters_hier = hier.fit_predict(X_scaled)
sil_hier = silhouette_score(X_scaled, clusters_hier)
db_hier = davies_bouldin_score(X_scaled, clusters_hier)

print(f"\nHierarchical:")
print(f"  Silhouette: {sil_hier:.4f}")
print(f"  Davies-Bouldin: {db_hier:.4f}")

df_unsup = pd.DataFrame({
    'Método': ['K-Means', 'Hierarchical'],
    'Silhouette': [sil_km, sil_hier],
    'Davies-Bouldin': [db_km, db_hier]
})

df_unsup.to_csv('weight_height_unsupervised_results.csv', index=False)
print("\n✓ Resultados salvos")

In [ ]:
# Visualização clustering
fig, axes = plt.subplots(1, 3, figsize=(20, 6))

# Real
for gender, color in zip([1, 0], ['blue', 'red']):
    mask = y == gender
    axes[0].scatter(X_scaled[mask, 0], X_scaled[mask, 1], 
                   c=color, alpha=0.4, s=10, label=['Male', 'Female'][1-gender])
axes[0].set_title('Classes Reais', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Height (normalizada)')
axes[0].set_ylabel('Weight (normalizada)')
axes[0].legend()
axes[0].grid(alpha=0.3)

# K-Means
scatter = axes[1].scatter(X_scaled[:, 0], X_scaled[:, 1], c=clusters_km, cmap='viridis', alpha=0.4, s=10)
axes[1].scatter(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:, 1],
               c='red', marker='X', s=300, edgecolors='black', linewidths=2, label='Centróides')
axes[1].set_title(f'K-Means\n(Silhouette: {sil_km:.3f})', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Height (normalizada)')
axes[1].set_ylabel('Weight (normalizada)')
axes[1].legend()
axes[1].grid(alpha=0.3)

# Hierarchical
axes[2].scatter(X_scaled[:, 0], X_scaled[:, 1], c=clusters_hier, cmap='viridis', alpha=0.4, s=10)
axes[2].set_title(f'Hierarchical\n(Silhouette: {sil_hier:.3f})', fontsize=14, fontweight='bold')
axes[2].set_xlabel('Height (normalizada)')
axes[2].set_ylabel('Weight (normalizada)')
axes[2].grid(alpha=0.3)

plt.tight_layout()
plt.savefig('weight_height_clustering_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Clustering visualizado")

## 4. Resumo Final

In [ ]:
print("="*60)
print("RESUMO - WEIGHT-HEIGHT DATASET")
print("="*60)

print("\n1. SUPERVISIONADO:")
best = df_results.loc[df_results['Accuracy'].idxmax()]
print(f"   Melhor: {best['Modelo']}")
print(f"   Accuracy: {best['Accuracy']:.4f}")
print(f"   F1-Score: {best['F1-Score']:.4f}")
above = (df_results['Accuracy'] >= 0.70).sum()
print(f"   Modelos >= 70%: {above}/{len(df_results)}")

print("\n2. NÃO SUPERVISIONADO:")
print(f"   K-Means Silhouette: {sil_km:.4f}")
print(f"   Hierarchical Silhouette: {sil_hier:.4f}")
print(f"   Melhor: {'K-Means' if sil_km > sil_hier else 'Hierarchical'}")

print("\n3. ARQUIVOS GERADOS:")
print("   ✓ weight_height_exploratory.png")
print("   ✓ weight_height_supervised_metrics.png")
print("   ✓ weight_height_clustering_comparison.png")
print("   ✓ weight_height_supervised_results.csv")
print("   ✓ weight_height_unsupervised_results.csv")

print("\n" + "="*60)
print("AVALIAÇÃO CONCLUÍDA")
print("="*60)